In [ ]:
import os
import numpy as np
import subprocess
from glob import glob
#import aplpy
from time import sleep
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.convolution import convolve
from astropy.convolution import Gaussian2DKernel
import pandas as pd
from tqdm import tqdm_notebook

# Add extra code to run things in parallel.
The combining exposure maps takes a REALLY long time. So I added this bit so I can run it all in parallel.

You don't need to run any of this if you don't want to run things in paralle. I'll try to comment out code at the bottom so you can run things in parallel or not depending on what you want to do.

In [ ]:
import multiprocessing
import traceback, functools

def error(msg, *args):
    multiprocessing.log_to_stderr()
    return multiprocessing.get_logger().error(msg, *args)

def trace_unhandled_exceptions(func):
    @functools.wraps(func)
    def wrapped_func(*args, **kwargs):
        try:
            func(*args, **kwargs)
        except Exception as e:
            error(traceback.format_exc())
            raise

    return wrapped_func

class AsyncFactory:
    def __init__(self, func, cb_func):
        self.func = func
        self.cb_func = cb_func
        self.pool = multiprocessing.Pool(maxtasksperchild=5,
                                         processes=multiprocessing.cpu_count())

    def call(self,*args, **kwargs):
        self.pool.apply_async(self.func, args, kwargs, self.cb_func)

    def wait(self):
        self.pool.close()
        self.pool.join()

def cb_func(f):
    print("PID: %d \t Value: %s completed" % (os.getpid(), f))

In [ ]:
def combineXRT(name, outpath):
    ''' This combines the individual observations
    
    IMPORTANT! The scripts that this (and other functions) creat are designed
    to be run from the same directory as this notebook. They WILL NOT work 
    if you try to run them from the individual data directories.
    
    '''
    
    # find the x-ray files
    files = glob(f'{outpath}/{name}/reduced/**/*xpcw3po_cl.evt', recursive=True)

    # sort the files so we can control their input order
    # this is used to make sure all of the data products are the same 
    # when we are combining observations
    
    files = np.sort(files)
    
    if len(files) < 1:
        return

    # write xsel.in
    with open(f'{outpath}/{name}/{name}_xsel.in', 'w') as f:
        for i, f_in in enumerate(files):
            f_parts = f_in.split('/')
            if i == 0:
                f.writelines('xsel\n')
                f.writelines('read events\n')
                # set the data directory
                f.writelines('/'.join(f_parts[:3]) + '\n')
                # first entry
                f.writelines('/'.join(f_parts[3:]) + '\n')
                f.writelines('yes\n')
                continue

            f.writelines('read events\n')
            f.writelines('/'.join(f_parts[3:]) + '\n')

        f.writelines('extract events\n')
        f.writelines(f'save events {"/".join(f_parts[:3])}/{name}_events.fits\n')
        f.writelines('yes\n')

                     
        f.writelines('set phaname PI\n')
        # here we are going to make a few binned images for a few different energy ranges
        # energies in loop
        for eng in [200, 300, 400, 500, 600]:
            f.writelines(f'filter pha_cutoff 50 {eng}\n')

            # save non-binned image -- the yes's are to overwrite if file is already there
            f.writelines('extract image\n')
            f.writelines(f'save image {"/".join(f_parts[:3])}/{name}_img_50-{eng}.fits\n')
            if os.path.isfile(f'{outpath}/{name}/{name}_img_50-{eng}.fits'):
                f.writelines('yes\n')

            # save binned image -- see above
            f.writelines('set xybinsize 8\n')
            f.writelines('extract image\n')
            f.writelines(f'save image {"/".join(f_parts[:3])}/{name}_img_50-{eng}_bl8.fits\n')
            if os.path.isfile(f'{outpath}/{name}/{name}_img_50-{eng}_bl8.fits'):
                f.writelines('yes\n')

            f.writelines('set xybinsize 4\n')
            f.writelines('extract image\n')
            f.writelines(f'save image {"/".join(f_parts[:3])}/{name}_img_50-{eng}_bl4.fits\n')
            if os.path.isfile(f'{outpath}/{name}/{name}_img_50-{eng}_bl4.fits'):
                f.writelines('yes\n')

        f.writelines('exit\n')
        f.writelines('no\n')

    # log the output
    log_file = f'{outpath}/{name}/{name}_xsel.log'
                         
    # call xselect
    os.system(f'xselect < {outpath}/{name}/{name}_xsel.in > {log_file}')

    return

In [ ]:
def combineXRT_exp(name, outpath):
    ''' This combines the exposure maps'''
    
    # find the x-ray files
    files = glob(f'{outpath}/{name}/reduced/**/*xpcw3po_ex.img', recursive=True)

    # sort the observations -- see above
    files = np.sort(files)
    
    if len(files) < 1:
        return name

    # remove the old file if it is there
    if os.path.isfile(f'{outpath}/{name}/{name}_exp.fits'):
        os.remove(f'{outpath}/{name}/{name}_exp.fits')

    # write xsel.in
    with open(f'{outpath}/{name}/{name}_ximg_exp.in', 'w') as f:
        for i, f_in in enumerate(files):
            f_parts = f_in.split('/')
            f.writelines(f'read {f_in}\n')
            if i == 0:
                continue
            f.writelines('sum\n')
            f.writelines('save\n')


        f.writelines(f'write/fits {"/".join(f_parts[:3])}/{name}_exp.fits\n')

        f.writelines('exit\n')
 
    # log the output
    log_file = f'{outpath}/{name}/{name}_ximg_exp.log'
    # call ximage
    os.system(f'ximage < {outpath}/{name}/{name}_ximg_exp.in > {log_file}')

    return name

In [ ]:
def load_PSZcatalog():
    from astropy.table import Table                                                       
    from numpy import append as npappend                                             

    datapath = './../planckClusters/catalogs/'
    
    ps1 = Table.read(f'{datapath}/PSZ1v2.1.fits')
    ps2 = Table.read(f'{datapath}/PSZ2v1.fits')

    # convert to pandas
    df1 = ps1.to_pandas()
    df2 = ps2.to_pandas()

    # clean up strings -- not required
    df1 = df1.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
    df2 = df2.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)

    # merge the catalogs together
    df_m = df1.merge(df2, how='outer', left_on='INDEX', right_on='PSZ', suffixes=('_PSZ1', '_PSZ2'))
    
    # get the columns that we want
    cols = df_m.columns[[0, 1, 4, 5, 8, 29, 33, 34, 37, 38, 40, 51]]
    df_final = df_m[cols]

    # remerge to find bits that were missing                                        
    df_final_bigger = df_final.merge(df2, how='left', left_on='INDEX_PSZ1',         
                                 right_on='PSZ')
    # fill in nans                                                                  
    for col in ['NAME', 'RA', 'DEC', 'SNR', 'REDSHIFT', 'INDEX']:                   
        df_final_bigger[col+'_PSZ2'] = df_final_bigger[col+'_PSZ2'].fillna(df_final_bigger[col])
    # fill in nans                                                                  
    for col in ['NAME', 'RA', 'DEC', 'SNR', 'REDSHIFT', 'INDEX']:
        df_final_bigger[col+'_PSZ2'] = df_final_bigger[col+'_PSZ2'].fillna(df_final_bigger[col])
    for col in ['NAME', 'RA', 'DEC']:
        df_final_bigger[col] = df_final_bigger[col+'_PSZ2'].fillna(df_final_bigger[col+'_PSZ1'])

    df_final_bigger = df_final_bigger[npappend(df_final_bigger.columns[:12].values, ['NAME', 'RA', 'DEC'])]

    return df_final_bigger


In [ ]:
# get file data
data = load_PSZcatalog()
data = data.sort_index(axis=1)

outpath = './data_full'

# this is the multitasking bit
async_worker = AsyncFactory(combineXRT_exp, cb_func)

#for i, (ra, dec, name) in enumerate(zip(data['RA'], data['DEC'], data['NAME'])):
for ra, dec, name in tqdm_notebook(zip(data['RA'], data['DEC'], data['NAME'])):
    
    #print(name)
    name = name.replace(' ', '_')

    #combineXRT(name, outpath)
    async_worker.call(name, outpath)
    #combineXRT_exp(name, outpath)

async_worker.wait()